<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Learning FourSquare API with Python</font></h1>

## Introduction

In this lab, you will learn in details how to make calls to the Foursquare API for different purposes. You will learn how to construct a URL to send a request to the API to search for a specific type of venues, to explore a particular venue, to explore a Foursquare user, to explore a geographical location, and to get trending venues around a location. Also, you will learn how to use the visualization library, Folium, to visualize the results.

## Table of Contents

1. <a href="#item1">Foursquare API Search Function</a>
2. <a href="#item2">Explore a Given Venue</a>  
3. <a href="#item3">Explore a User</a>  
4. <a href="#item4">Foursquare API Explore Function</a>  
5. <a href="#item5">Get Trending Venues</a>  

### Import necessary Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.6                |           py36_0         872 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         872 KB

The following packages will be UPDATED:

  conda                                        4.6.4-py36_0 --> 4.6.6-py36_0



conda-4.6.6          | 872 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Folium installed


### Define Foursquare Credentials and Version

##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [2]:
CLIENT_ID = 'JAH2Y0HES01BIMCMSRULSHTXIVUYLOQPGNJBIC0LWJ4KWMUL' # your Foursquare ID
CLIENT_SECRET = 'GEZ1UJJVLK5YIXKZPMG03UBNBYHMJPA52NAEDO1GSP5IL1PA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JAH2Y0HES01BIMCMSRULSHTXIVUYLOQPGNJBIC0LWJ4KWMUL
CLIENT_SECRET:GEZ1UJJVLK5YIXKZPMG03UBNBYHMJPA52NAEDO1GSP5IL1PA


#### Let's again assume that you are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [3]:
address = 'Newark, NJ'
CID = '4bf58dd8d48988d13d941735'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.735657 -74.1723667


<a id="item1"></a>

## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

#### Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel. 

In [4]:
search_query = 'High School'
radius = 2000
print(search_query + ' .... OK!')

High School .... OK!


#### Define the corresponding URL

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&categoryId={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, CID, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=JAH2Y0HES01BIMCMSRULSHTXIVUYLOQPGNJBIC0LWJ4KWMUL&client_secret=GEZ1UJJVLK5YIXKZPMG03UBNBYHMJPA52NAEDO1GSP5IL1PA&ll=40.735657,-74.1723667&v=20180604&query=High School&radius=2000&categoryId=4bf58dd8d48988d13d941735&limit=1000'

#### Send the GET Request and examine the results

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c6f1a4f6a60714c8521df0c'},
 'response': {'venues': [{'id': '4ba0c7a3f964a520257d37e3',
    'name': 'East Side High School',
    'location': {'address': '238 Van Buren St',
     'crossStreet': 'Nichols St',
     'lat': 40.724079383790375,
     'lng': -74.15865215808857,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.724079383790375,
       'lng': -74.15865215808857}],
     'distance': 1731,
     'postalCode': '07105',
     'cc': 'US',
     'city': 'Newark',
     'state': 'NJ',
     'country': 'United States',
     'formattedAddress': ['238 Van Buren St (Nichols St)',
      'Newark, NJ 07105',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d13d941735',
      'name': 'High School',
      'pluralName': 'High Schools',
      'shortName': 'High School',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1550785103',
 

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",False,4ba0c7a3f964a520257d37e3,238 Van Buren St,US,Newark,United States,Nichols St,1731,"[238 Van Buren St (Nichols St), Newark, NJ 071...","[{'label': 'display', 'lat': 40.72407938379037...",40.724079,-74.158652,07105,NJ,East Side High School,v-1550785103,NaN
1,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",False,4bd575b75631c9b6b90ea430,550 Dr Martin Luther King Jr Blvd,US,Newark,United States,NaN,619,"[550 Dr Martin Luther King Jr Blvd, Newark, NJ...","[{'label': 'display', 'lat': 40.73412369252835...",40.734124,-74.179429,07102,NJ,Arts High School,v-1550785103,NaN
2,"[{'id': '4bf58dd8d48988d1af941735', 'name': 'C...",False,4bd0e42041b9ef3b3416fbe5,246 18th Ave,US,Newark,United States,NaN,1834,"[246 18th Ave, Newark, NJ 07108, United States]","[{'label': 'display', 'lat': 40.73031653934087...",40.730317,-74.192937,07108,NJ,Central High School,v-1550785103,NaN
3,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",False,4bdc474363c5c9b677082b68,260 Norfolk St,US,Newark,United States,NaN,1248,"[260 Norfolk St, Newark, NJ 07103, United States]","[{'label': 'display', 'lat': 40.74034647748234...",40.740346,-74.185811,07103,NJ,Science High School,v-1550785103,NaN
4,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",False,4d0cdf29257d6dcbe4809867,238 Van Buren St,US,Newark,United States,NaN,1538,"[238 Van Buren St, Newark, NJ 07105, United St...","[{'label': 'display', 'lat': 40.72465512585857...",40.724655,-74.161329,07105,NJ,East Side High School,v-1550785103,NaN


#### Define information of interest and filter dataframe

In [8]:
dataframe
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,East Side High School,High School,238 Van Buren St,US,Newark,United States,Nichols St,1731,"[238 Van Buren St (Nichols St), Newark, NJ 071...","[{'label': 'display', 'lat': 40.72407938379037...",40.724079,-74.158652,07105,NJ,4ba0c7a3f964a520257d37e3
1,Arts High School,High School,550 Dr Martin Luther King Jr Blvd,US,Newark,United States,NaN,619,"[550 Dr Martin Luther King Jr Blvd, Newark, NJ...","[{'label': 'display', 'lat': 40.73412369252835...",40.734124,-74.179429,07102,NJ,4bd575b75631c9b6b90ea430
2,Central High School,College Auditorium,246 18th Ave,US,Newark,United States,NaN,1834,"[246 18th Ave, Newark, NJ 07108, United States]","[{'label': 'display', 'lat': 40.73031653934087...",40.730317,-74.192937,07108,NJ,4bd0e42041b9ef3b3416fbe5
3,Science High School,High School,260 Norfolk St,US,Newark,United States,NaN,1248,"[260 Norfolk St, Newark, NJ 07103, United States]","[{'label': 'display', 'lat': 40.74034647748234...",40.740346,-74.185811,07103,NJ,4bdc474363c5c9b677082b68
4,East Side High School,High School,238 Van Buren St,US,Newark,United States,NaN,1538,"[238 Van Buren St, Newark, NJ 07105, United St...","[{'label': 'display', 'lat': 40.72465512585857...",40.724655,-74.161329,07105,NJ,4d0cdf29257d6dcbe4809867
5,High School,High School,1 Newark Ctr,US,Newark,United States,NaN,528,"[1 Newark Ctr, Newark, NJ 07102, United States]","[{'label': 'display', 'lat': 40.736565, 'lng':...",40.736565,-74.166214,07102,NJ,4c746ec144d3952113ccf4be
6,Marion P. Thomas High School of Culinary and P...,High School,125 Sussex Avenue,US,Newark,United States,NaN,1402,"[125 Sussex Avenue, Newark, NJ 07103, United S...","[{'label': 'display', 'lat': 40.74661587015097...",40.746616,-74.180582,07103,NJ,5668398a498e1ede14923851
7,Bard High School Early College,College Academic Building,321 Bergen St,US,Newark,United States,NaN,1917,"[321 Bergen St, Newark, NJ 07103, United States]","[{'label': 'display', 'lat': 40.73478571256717...",40.734786,-74.195075,07103,NJ,4e5baea28877c8a76e021cdc
8,Arts High school,High School,NaN,US,Newark,United States,NaN,675,"[Newark, NJ 07102, United States]","[{'label': 'display', 'lat': 40.73609, 'lng': ...",40.736090,-74.180360,07102,NJ,4c1b9d8363750f473d22b567
9,Visions Academy Charter High School,High School,NaN,US,Newark,United States,NaN,731,"[Newark, NJ 07102, United States]","[{'label': 'display', 'lat': 40.73280557895605...",40.732806,-74.180174,07102,NJ,4f4cd602e4b0f92023eeffdf


#### Let's visualize the Italian restaurants that are nearby

In [9]:
dataframe_filtered.name

0                                 East Side High School
1                                      Arts High School
2                                   Central High School
3                                   Science High School
4                                 East Side High School
5                                           High School
6     Marion P. Thomas High School of Culinary and P...
7                        Bard High School Early College
8                                      Arts High school
9                   Visions Academy Charter High School
10                                Barringer High School
11                        Newark Vocational High School
12                 Marion P. Thomas Charter High School
13                     Newark Early College High School
14                                Barringer High School
15          North Star Academy Charter School of Newark
16                           Newark Prep Charter School
17        The Paulo Freire School aka The Freire